<a href="https://colab.research.google.com/github/Abhishek-7504/PDF-Summariser/blob/main/PDF_summarizer_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2 transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00


In [ ]:
# 1) Install dependencies
!pip install --quiet PyPDF2 transformers torch

# 2) Imports
import PyPDF2
from transformers import pipeline
from tqdm.notebook import tqdm

# 3) PDF → text
def extract_text_from_pdf(path):
    reader = PyPDF2.PdfReader(path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() or ""
    return text

# 4) Split into chunks of ~max_len chars
def split_text(text, max_len=1000):
    words = text.split()
    chunks, curr = [], ""
    for w in words:
        if len(curr) + len(w) + 1 > max_len:
            chunks.append(curr)
            curr = w
        else:
            curr += (" " + w) if curr else w
    if curr:
        chunks.append(curr)
    return chunks

# 5) Init the distilled summarizer on GPU
summarizer = pipeline(
    "summarization",
    model="sshleifer/distilbart-cnn-12-6",
    device=0,               # 0 = first GPU (your T4)
    framework="pt"          # ensure PyTorch backend
)

# 6) Load PDF, chunk & summarize
pdf_path = "dbms.pdf"        # change to your filename
raw = extract_text_from_pdf(pdf_path)
chunks = split_text(raw, max_len=1000)

# 7) Summarize each chunk with a progress bar
summaries = []
for chunk in tqdm(chunks, desc="Summarizing"):
    out = summarizer(
        chunk,
        max_length=150,
        min_length=40,
        do_sample=False
    )
    summaries.append(out[0]["summary_text"])

# 8) Combine, print, and save
final_summary = "\n".join(summaries)
print("\n--- FINAL SUMMARY ---\n")
print(final_summary)

with open("summary_output.txt", "w", encoding="utf-8") as f:
    f.write(final_summary)

print("\nSaved to summary_output.txt")


Device set to use cuda:0


Summarizing:   0%|          | 0/5 [00:00<?, ?it/s]


--- FINAL SUMMARY ---

 A Relation Schema defines the structure of a database table, including its attributes and constraints . Good Schema design reduces data redundancy, prevents anomalies and improves query performance . Poor Schema Design leads to inconsistent data, inefficiencies, and complex queries .
 Too many NULL values make queries complex and inefficient . NULL values indicate missing or inapplicable data . Decomposing tables incorrectly can break functional dependencies . This can lead to inconsistent or incorrect data retrieval .
 Armstrong’s Axioms & Derived Rules in Database Design . Inference Rules are rules used to derive all possible functional dependencies from a given set . They are fundamental to: ●Normalization, integrity enforcement, and candidate key discovery .
 A minimal cover of a set of functional dependencies (FDs) F is an equivalent set of FDs G that: 1. Has the same closure as F F, G, F, CF, G . A minimum cover is a minimal cover (Canonical Cover)
 Given